In [1]:
import pandas as pd

In [7]:
df= pd.read_csv('../01.Data_Collection/screen_show_seat.csv')

In [8]:
df.shape

(70845, 19)

In [9]:
df.head()

,영화명,개봉일,관객수,누적관객수,매출액,누적매출액,상영_순위,상영점유율,상영횟수,상영_점유율집계일,좌석_순위,좌석판매율,좌석점유율,좌석수,좌석_점유율집계일,스크린_순위,스크린점유율,스크린수,스크린_점유율집계일
0,국제시장,2014-12-17,188721.0,8362611.0,1.398815e+09,6.504937e+10,1.0,53.092%,4164.0,2015-01-07,1.0,21.5%,34.7%,878197.0,2015-01-01,1.0,38.9%,838.0,2015-01-01
1,"님아, 그 강을 건너지 마오",2014-11-27,29657.0,4478450.0,2.157727e+08,3.489502e+10,2.0,17.226%,1351.0,2015-01-07,2.0,16.9%,6.95%,175899.0,2015-01-01,2.0,19.9%,428.0,2015-01-01
2,기술자들,2014-12-24,24046.0,2360507.0,1.836565e+08,1.826173e+10,4.0,12.176%,955.0,2015-01-07,3.0,18.2%,5.21%,131812.0,2015-01-01,3.0,15.5%,335.0,2015-01-01
3,워킹걸,2015-01-07,14281.0,18503.0,1.109452e+08,1.437702e+08,3.0,12.291%,964.0,2015-01-07,4.0,11.1%,5.07%,128267.0,2015-01-01,4.0,13.7%,296.0,2015-01-01
4,개를 훔치는 완벽한 방법,2014-12-31,4262.0,183398.0,3.120990e+07,1.393379e+09,5.0,2.308%,181.0,2015-01-07,5.0,19.2%,0.88%,22259.0,2015-01-01,5.0,4.7%,102.0,2015-01-01
